In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.6 MB/s eta 0:00:00


Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Training

In [ ]:
from ultralytics import YOLO
import torch

# Check if GPU is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

model = YOLO('yolo11n.pt')

results = model.train(
    data='/content/drive/MyDrive/School/yasmine/dataset/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    save=True,
    device=0, #GPU Nvidia T4 from google colab
    project='seatbelt_detection',
    name='yolo11n_run1',

    # Data augmentation
    hsv_h=0.015,                 # HSV-Hue augmentation
    hsv_s=0.7,                   # HSV-Saturation augmentation
    hsv_v=0.4,                   # HSV-Value augmentation
    degrees=10,                  # Rotation augmentation
    translate=0.1,               # Translation augmentation
    scale=0.5,                   # Scale augmentation
    shear=0.0,                   # Shear augmentation
    perspective=0.0,             # Perspective augmentation
    flipud=0.0,                  # Vertical flip probability
    fliplr=0.5,                  # Horizontal flip probability
    mosaic=1.0,                  # Mosaic augmentation probability
    mixup=0.0,                   # Mixup augmentation probability

    # Optimization
    optimizer='Adam',            # Optimizer (Adam, SGD, AdamW)
    lr0=0.01,                    # Initial learning rate
    lrf=0.01,                    # Final learning rate factor
    momentum=0.937,              # Momentum
    weight_decay=0.0005,         # Weight decay

    # Performance tuning
    workers=8,                   # Number of worker threads for data loading
    cache=False,                 # Cache images for faster training (uses more RAM)

    plots=True,                  # Generate training plots
    verbose=True                 # Verbose output
)

print(f"Results saved in: {results.save_dir}")

CUDA available: True
GPU: Tesla T4
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/School/yasmine/dataset/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_run12, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, o

Evaluating

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# Load your trained model
model = YOLO('/content/seatbelt_detection/yolo11n_run12/weights/best.pt')

print("EVALUATING MODEL ON TEST SET")

# Evaluate on test set
metrics = model.val(
    data='/content/drive/MyDrive/School/yasmine/dataset/data.yaml',
    split='test',
    imgsz=640,
    batch=16,
    save_json=True,
    plots=True
)

print("EVALUATION RESULTS")

# Overall metrics
print(f"\nmAP50 (IoU=0.5): {metrics.box.map50:.4f}")
print(f"mAP50-95 (IoU=0.5:0.95): {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

# Per-class metrics
print("\nPer-Class Performance:")
print("-" * 70)
class_names = ['person_no_seatbelt', 'person_seatbelt']

for i, class_name in enumerate(class_names):
    if i < len(metrics.box.ap):
        print(f"{class_name}:")
        print(f"  AP50: {metrics.box.ap50[i]:.4f}")
        print(f"  AP50-95: {metrics.box.ap[i]:.4f}")

print("\nWhat these metrics mean for real-world deployment:")
print(f"- Precision ({metrics.box.mp:.2%}): When model says 'no seatbelt', it's correct {metrics.box.mp:.2%} of the time")
print(f"- Recall ({metrics.box.mr:.2%}): Model catches {metrics.box.mr:.2%} of actual violations")
print(f"- mAP50 ({metrics.box.map50:.2%}): Overall detection quality (higher is better)")

if metrics.box.mp > 0.9:
    print("\n HIGH PRECISION: Few false alarms - good for enforcement systems")
elif metrics.box.mp > 0.7:
    print("\n MODERATE PRECISION: Some false alarms - may need human verification")
else:
    print("\n LOW PRECISION: Many false alarms - needs improvement")

if metrics.box.mr > 0.9:
    print("HIGH RECALL: Catches most violations - good for safety monitoring")
elif metrics.box.mr > 0.7:
    print("MODERATE RECALL: Misses some violations - acceptable for warnings")
else:
    print("LOW RECALL: Misses many violations - needs improvement")

print("Check for:")
print("  - confusion_matrix.png")
print("  - PR_curve.png")
print("  - F1_curve.png")

EVALUATING MODEL ON TEST SET
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 28.2±15.4 MB/s, size: 47.2 KB)
val: Scanning /content/drive/MyDrive/School/yasmine/dataset/test/labels.cache... 557 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 557/557 773.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 3.4it/s 10.3s
                   all        557        729      0.798      0.562      0.666      0.342
    person_no_seatbelt        299        348      0.824      0.486      0.614       0.31
       person_seatbelt        330        381      0.772      0.638      0.718      0.374
Speed: 2.1ms preprocess, 4.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Saving /content/runs/detect/val9/predictions.json...
Results saved to /content/runs/de

Test on Real Images/Videos

In [ ]:
from ultralytics import YOLO
import cv2
from pathlib import Path
import time

# Load trained model
model = YOLO('/content/seatbelt_detection/yolo11n_run12/weights/best.pt')

def predict_images(image_path, conf_threshold=0.5):
    print(f"\nRunning inference on: {image_path}")
    print(f"Confidence threshold: {conf_threshold}")

    results = model.predict(
        source=image_path,
        conf=conf_threshold,      # Confidence threshold
        iou=0.5,                  # NMS IoU threshold
        imgsz=640,                # Image size
        save=True,                # Save results with bounding boxes
        save_txt=True,            # Save results as txt files
        save_conf=True,           # Save confidence scores
        show_labels=True,         # Show labels on images
        show_conf=True,           # Show confidence on images
        line_width=2,             # Bounding box thickness
        project='inference_results',
        name='images'
    )

    # Analyze results
    total_violations = 0
    total_compliant = 0

    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = model.names[cls]

            if 'no_seatbelt' in class_name or 'noseatbelt' in class_name:
                total_violations += 1
            else:
                total_compliant += 1

    print("DETECTION SUMMARY:")
    print(f"  Violations (no seatbelt): {total_violations}")
    print(f"  Compliant (seatbelt worn): {total_compliant}")
    print(f"  Total detections: {total_violations + total_compliant}")
    print(f"Results saved in: inference_results/images/")

if __name__ == "__main__":
    print("\n[1] Testing on images...")
    predict_images(
        image_path='/content/drive/MyDrive/School/yasmine/dataset/test/images/',
        conf_threshold=0.5
    )

SEATBELT DETECTION - INFERENCE MODE

[1] Testing on images...

Running inference on: /content/drive/MyDrive/School/yasmine/dataset/test/images/
Confidence threshold: 0.5

image 1/557 /content/drive/MyDrive/School/yasmine/dataset/test/images/dataset1_1625227980267_jpg.rf.92996181c7d73e3b8f330d9b12b95e53.jpg: 640x640 1 person_seatbelt, 10.2ms
image 2/557 /content/drive/MyDrive/School/yasmine/dataset/test/images/dataset1_1625227980304_jpg.rf.c9f55b044b1a0d1093bc9543f684f35d.jpg: 640x640 1 person_seatbelt, 15.5ms
image 3/557 /content/drive/MyDrive/School/yasmine/dataset/test/images/dataset1_1625227980306_jpg.rf.438f711355f4c286cd464b0e1538835e.jpg: 640x640 1 person_seatbelt, 12.5ms
image 4/557 /content/drive/MyDrive/School/yasmine/dataset/test/images/dataset1_1625227980311_jpg.rf.f43e64ab864dc5457fd43b395b86ef10.jpg: 640x640 2 person_no_seatbelts, 9.8ms
image 5/557 /content/drive/MyDrive/School/yasmine/dataset/test/images/dataset1_1625227980351_jpg.rf.8e187e1cd585d8e3fb6c8169f453860a.jpg: 